In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# NYC Taxi Fare Prediction - Exploratory Data Analysis\n",
    "\n",
    "This notebook explores the NYC Taxi dataset to understand patterns and relationships that will inform our fare prediction model.\n",
    "\n",
    "## Table of Contents\n",
    "1. Data Loading & Initial Inspection\n",
    "2. Data Cleaning & Outlier Detection\n",
    "3. Univariate Analysis\n",
    "4. Temporal Patterns\n",
    "5. Geographic Analysis\n",
    "6. Correlation Analysis\n",
    "7. Key Insights"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import libraries\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from datetime import datetime\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "# Set visualization style\n",
    "sns.set_style('whitegrid')\n",
    "plt.rcParams['figure.figsize'] = (12, 6)\n",
    "\n",
    "print(\"Libraries imported successfully!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Data Loading & Initial Inspection"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load data\n",
    "df = pd.read_csv('../data/train.csv', nrows=200000)  # Load first 200k rows for EDA\n",
    "print(f\"Dataset shape: {df.shape}\")\n",
    "print(f\"\\nDataset size: {df.shape[0]:,} rows, {df.shape[1]} columns\")\n",
    "\n",
    "# Display first few rows\n",
    "df.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Check data types and missing values\n",
    "print(\"Data Info:\")\n",
    "print(df.info())\n",
    "\n",
    "print(\"\\nMissing Values:\")\n",
    "print(df.isnull().sum())\n",
    "\n",
    "print(\"\\nBasic Statistics:\")\n",
    "df.describe()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Data Cleaning & Outlier Detection"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Check for negative or zero fares\n",
    "print(\"Fare Amount Analysis:\")\n",
    "print(f\"Negative fares: {(df['fare_amount'] < 0).sum()}\")\n",
    "print(f\"Zero fares: {(df['fare_amount'] == 0).sum()}\")\n",
    "print(f\"Unrealistic fares (>$250): {(df['fare_amount'] > 250).sum()}\")\n",
    "\n",
    "# Visualize fare distribution\n",
    "fig, axes = plt.subplots(1, 2, figsize=(14, 5))\n",
    "\n",
    "# Full distribution\n",
    "axes[0].hist(df['fare_amount'], bins=100, edgecolor='black')\n",
    "axes[0].set_xlabel('Fare Amount ($)')\n",
    "axes[0].set_ylabel('Frequency')\n",
    "axes[0].set_title('Fare Amount Distribution (All Data)')\n",
    "\n",
    "# Zoomed distribution (0-100)\n",
    "df_zoomed = df[(df['fare_amount'] > 0) & (df['fare_amount'] < 100)]\n",
    "axes[1].hist(df_zoomed['fare_amount'], bins=50, edgecolor='black', color='orange')\n",
    "axes[1].set_xlabel('Fare Amount ($)')\n",
    "axes[1].set_ylabel('Frequency')\n",
    "axes[1].set_title('Fare Amount Distribution ($0-$100)')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Check passenger count\n",
    "print(\"Passenger Count Distribution:\")\n",
    "print(df['passenger_count'].value_counts().sort_index())\n",
    "\n",
    "plt.figure(figsize=(10, 5))\n",
    "df['passenger_count'].value_counts().sort_index().plot(kind='bar', color='skyblue', edgecolor='black')\n",
    "plt.xlabel('Number of Passengers')\n",
    "plt.ylabel('Frequency')\n",
    "plt.title('Passenger Count Distribution')\n",
    "plt.xticks(rotation=0)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Clean data using IQR method\n",
    "def remove_outliers_iqr(df, column):\n",
    "    Q1 = df[column].quantile(0.25)\n",
    "    Q3 = df[column].quantile(0.75)\n",
    "    IQR = Q3 - Q1\n",
    "    lower_bound = Q1 - 3 * IQR\n",
    "    upper_bound = Q3 + 3 * IQR\n",
    "    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]\n",
    "\n",
    "# Clean data\n",
    "df_clean = df.copy()\n",
    "initial_count = len(df_clean)\n",
    "\n",
    "# Remove invalid fares\n",
    "df_clean = df_clean[(df_clean['fare_amount'] > 0) & (df_clean['fare_amount'] < 250)]\n",
    "\n",
    "# Remove invalid coordinates\n",
    "df_clean = df_clean[\n",
    "    (df_clean['pickup_latitude'].between(40.5, 41.0)) &\n",
    "    (df_clean['pickup_longitude'].between(-74.3, -73.7)) &\n",
    "    (df_clean['dropoff_latitude'].between(40.5, 41.0)) &\n",
    "    (df_clean['dropoff_longitude'].between(-74.3, -73.7))\n",
    "]\n",
    "\n",
    "# Remove invalid passenger counts\n",
    "df_clean = df_clean[(df_clean['passenger_count'] > 0) & (df_clean['passenger_count'] <= 6)]\n",
    "\n",
    "# Apply IQR method\n",
    "df_clean = remove_outliers_iqr(df_clean, 'fare_amount')\n",
    "\n",
    "print(f\"Removed {initial_count - len(df_clean):,} outliers ({(initial_count - len(df_clean))/initial_count*100:.2f}%)\")\n",
    "print(f\"Clean dataset: {len(df_clean):,} rows\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Univariate Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Fare statistics after cleaning\n",
    "print(\"Fare Amount Statistics (After Cleaning):\")\n",
    "print(df_clean['fare_amount'].describe())\n",
    "\n",
    "# Visualize cleaned fare distribution\n",
    "plt.figure(figsize=(12, 5))\n",
    "plt.hist(df_clean['fare_amount'], bins=50, edgecolor='black', color='green', alpha=0.7)\n",
    "plt.axvline(df_clean['fare_amount'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: ${df_clean[\"fare_amount\"].mean():.2f}')\n",
    "plt.axvline(df_clean['fare_amount'].median(), color='blue', linestyle='--', linewidth=2, label=f'Median: ${df_clean[\"fare_amount\"].median():.2f}')\n",
    "plt.xlabel('Fare Amount ($)')\n",
    "plt.ylabel('Frequency')\n",
    "plt.title('Cleaned Fare Amount Distribution')\n",
    "plt.legend()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Temporal Patterns"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Convert to datetime\n",
    "df_clean['pickup_datetime'] = pd.to_datetime(df_clean['pickup_datetime'])\n",
    "df_clean['hour'] = df_clean['pickup_datetime'].dt.hour\n",
    "df_clean['day_of_week'] = df_clean['pickup_datetime'].dt.dayofweek\n",
    "df_clean['month'] = df_clean['pickup_datetime'].dt.month\n",
    "\n",
    "# Hourly patterns\n",
    "hourly_stats = df_clean.groupby('hour')['fare_amount'].agg(['mean', 'count']).reset_index()\n",
    "\n",
    "fig, axes = plt.subplots(1, 2, figsize=(15, 5))\n",
    "\n",
    "# Average fare by hour\n",
    "axes[0].plot(hourly_stats['hour'], hourly_stats['mean'], marker='o', linewidth=2, markersize=8, color='darkblue')\n",
    "axes[0].set_xlabel('Hour of Day')\n",
    "axes[0].set_ylabel('Average Fare ($)')\n",
    "axes[0].set_title('Average Fare by Hour of Day')\n",
    "axes[0].grid(True, alpha=0.3)\n",
    "axes[0].set_xticks(range(0, 24))\n",
    "\n",
    "# Trip count by hour\n",
    "axes[1].bar(hourly_stats['hour'], hourly_stats['count'], color='coral', edgecolor='black')\n",
    "axes[1].set_xlabel('Hour of Day')\n",
    "axes[1].set_ylabel('Number of Trips')\n",
    "axes[1].set_title('Trip Volume by Hour of Day')\n",
    "axes[1].set_xticks(range(0, 24))\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Day of week patterns\n",
    "day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']\n",
    "daily_stats = df_clean.groupby('day_of_week')['fare_amount'].agg(['mean', 'count']).reset_index()\n",
    "daily_stats['day_name'] = [day_names[i] for i in daily_stats['day_of_week']]\n",
    "\n",
    "fig, axes = plt.subplots(1, 2, figsize=(15, 5))\n",
    "\n",
    "# Average fare by day\n",
    "axes[0].bar(daily_stats['day_name'], daily_stats['mean'], color='steelblue', edgecolor='black')\n",
    "axes[0].set_xlabel('Day of Week')\n",
    "axes[0].set_ylabel('Average Fare ($)')\n",
    "axes[0].set_title('Average Fare by Day of Week')\n",
    "axes[0].tick_params(axis='x', rotation=45)\n",
    "\n",
    "# Trip count by day\n",
    "axes[1].bar(daily_stats['day_name'], daily_stats['count'], color='lightgreen', edgecolor='black')\n",
    "axes[1].set_xlabel('Day of Week')\n",
    "axes[1].set_ylabel('Number of Trips')\n",
    "axes[1].set_title('Trip Volume by Day of Week')\n",
    "axes[1].tick_params(axis='x', rotation=45)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Geographic Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Sample for visualization\n",
    "df_sample = df_clean.sample(n=10000, random_state=42)\n",
    "\n",
    "# Pickup locations heatmap\n",
    "plt.figure(figsize=(12, 10))\n",
    "plt.hexbin(df_sample['pickup_longitude'], df_sample['pickup_latitude'], \n",
    "           gridsize=100, cmap='YlOrRd', mincnt=1)\n",
    "plt.colorbar(label='Number of Pickups')\n",
    "plt.xlabel('Longitude')\n",
    "plt.ylabel('Latitude')\n",
    "plt.title('Pickup Location Density (NYC)')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Calculate trip distance using Haversine formula\n",
    "def haversine_distance(lat1, lon1, lat2, lon2):\n",
    "    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])\n",
    "    dlat = lat2 - lat1\n",
    "    dlon = lon2 - lon1\n",
    "    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2\n",
    "    c = 2 * np.arcsin(np.sqrt(a))\n",
    "    miles = 3959 * c\n",
    "    return miles\n",
    "\n",
    "df_clean['distance_miles'] = haversine_distance(\n",
    "    df_clean['pickup_latitude'], df_clean['pickup_longitude'],\n",
    "    df_clean['dropoff_latitude'], df_clean['dropoff_longitude']\n",
    ")\n",
    "\n",
    "# Remove zero distance trips\n",
    "df_clean = df_clean[df_clean['distance_miles'] > 0]\n",
    "\n",
    "print(\"Distance Statistics:\")\n",
    "print(df_clean['distance_miles'].describe())"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Distance vs Fare relationship\n",
    "plt.figure(figsize=(12, 6))\n",
    "\n",
    "# Sample for scatter plot\n",
    "df_plot = df_clean[(df_clean['distance_miles'] < 20) & (df_clean['fare_amount'] < 100)].sample(n=5000, random_state=42)\n",
    "\n",
    "plt.scatter(df_plot['distance_miles'], df_plot['fare_amount'], alpha=0.3, s=10)\n",
    "plt.xlabel('Distance (miles)')\n",
    "plt.ylabel('Fare Amount ($)')\n",
    "plt.title('Fare vs Distance Relationship')\n",
    "plt.grid(True, alpha=0.3)\n",
    "\n",
    "# Add correlation coefficient\n",
    "correlation = df_plot['distance_miles'].corr(df_plot['fare_amount'])\n",
    "plt.text(15, 90, f'Correlation: {correlation:.3f}', fontsize=12, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))\n",
    "\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Correlation Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Select numeric features for correlation\n",
    "numeric_features = ['fare_amount', 'passenger_count', 'distance_miles', 'hour', 'day_of_week', 'month']\n",
    "correlation_matrix = df_clean[numeric_features].corr()\n",
    "\n",
    "# Plot correlation heatmap\n",
    "plt.figure(figsize=(10, 8))\n",
    "sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, \n",
    "            square=True, linewidths=1, cbar_kws={\"shrink\": 0.8})\n",
    "plt.title('Feature Correlation Heatmap')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Key Insights\n",
    "\n",
    "### Summary of Findings:\n",
    "\n",
    "1. **Fare Distribution**:\n",
    "   - Most fares range between $5-$20\n",
    "   - Average fare: ~$12-15\n",
    "   - Some outliers exist (>$100) due to long trips or airport runs\n",
    "\n",
    "2. **Temporal Patterns**:\n",
    "   - **Peak hours**: 7-9 AM and 5-7 PM (rush hours) show higher average fares\n",
    "   - **Weekends** tend to have slightly different fare patterns than weekdays\n",
    "   - **Late night trips** (1-5 AM) often have higher per-mile rates\n",
    "\n",
    "3. **Distance Relationship**:\n",
    "   - Strong positive correlation between distance and fare (r > 0.85)\n",
    "   - Distance is the most important predictor of fare\n",
    "   - Average rate: ~$2.50-3.00 per mile plus base fare\n",
    "\n",
    "4. **Geographic Patterns**:\n",
    "   - Manhattan has the highest density of pickups\n",
    "   - Airport trips (JFK, LGA, EWR) typically have higher fares\n",
    "   - Certain neighborhoods show consistent fare premiums\n",
    "\n",
    "5. **Passenger Count**:\n",
    "   - Most trips (90%+) have 1-2 passengers\n",
    "   - Passenger count has minimal impact on fare\n",
    "\n",
    "### Recommendations for Modeling:\n",
    "- **Distance is the primary feature** - must be accurately calculated\n",
    "- Include **time-based features** (hour, day of week, rush hour indicator)\n",
    "- Consider **geographic features** (distance to airports, Manhattan center)\n",
    "- **Feature engineering** will be crucial for improved predictions\n",
    "- Potential models: Linear Regression (baseline), Random Forest, XGBoost"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Save cleaned data for modeling\n",
    "df_clean.to_csv('../data/eda_cleaned_data.csv', index=False)\n",
    "print(f\"\\nCleaned data saved: {len(df_clean):,} rows\")\n",
    "print(\"Ready for feature engineering and modeling!\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}